In [3]:
import os 
import numpy as np
import pandas as pd

In [8]:
import json

In [70]:
path_preds = "/network/projects/_groups/ecosystem-embeddings/ebird_checkpoints/resnet18_location_300_epochs/preds/"

path_val = "/ecosystem-embedding/training/valid_clustered_vf.csv"

valid = pd.read_csv(path_val)

scores_df = valid[["hotspot"]].copy()

scores_df["mae"]=0
scores_df["mse"]=0
scores_df["ce"]=0
scores_df["topk"]=0

In [65]:
not_idx = np.load('/network/projects/_groups/ecosystem-embeddings/species_splits/not_songbirds_idx.npy')
def get_species(df, hotspot):
    path = df.loc[df["hotspot"] == hotspot]["species"].values[0]
    with open(path, "r") as f:
        data = json.load(f)
        f.close()
    
    return(np.array(data["probs"])[not_idx])

In [66]:
def get_preds(path_preds, hotspot):
    return(np.load(os.path.join(path_preds, hotspot + ".npy")))

In [72]:
def topk(preds, target):
    
    k = np.count_nonzero(target)
      
    i_pred = np.argpartition(preds, -k)[-k:]

    i_targ= np.argpartition(target, -k)[-k:]

    if k == 0 :
        correct = 1
    else:
        correct = len(set(i_pred).intersection(set(i_targ))) / k
    return(correct)
        


In [81]:
for i, row in scores_df.iterrows():
    gt = get_species(valid, row["hotspot"])
    preds = get_preds(path_preds, row["hotspot"])
    
    scores_df.loc[i, 'mae'] = np.mean(np.abs(gt - preds))
    scores_df.loc[i, 'mse'] = np.mean(np.abs(gt - preds)**2)
    scores_df.loc[i, "topk"] = topk(preds, gt)

In [82]:
scores_df.mean()

mae     0.012156
mse     0.003451
ce      0.000000
topk    0.603227
dtype: float64

In [79]:
path_preds = "/network/projects/_groups/ecosystem-embeddings/ebird_checkpoints/resnet18_baseline/preds/"

path_val = "/ecosystem-embedding/training/valid_clustered_vf.csv"

valid = pd.read_csv(path_val)

scores_df_base = valid[["hotspot"]].copy()

scores_df_base["mae"]=0
scores_df_base["mse"]=0
scores_df_base["ce"]=0
scores_df_base["topk"]=0

for i, row in scores_df_base.iterrows():
    gt = get_species(valid, row["hotspot"])
    preds = get_preds(path_preds, row["hotspot"])
    
    scores_df_base.loc[i, 'mae'] = np.mean(np.abs(gt - preds))
    scores_df_base.loc[i, 'mse'] = np.mean(np.abs(gt - preds)**2)
    scores_df_base.loc[i, "topk"] = topk(preds, gt)

In [80]:
scores_df_base.mean()

mae     0.012156
mse     0.003451
ce      0.000000
topk    0.603227
dtype: float64